# Demo with atlas v1 fixes and compression levels

See discussion about compression levels:
https://github.com/PCMDI/cmor/issues/403

In [1]:
import xarray as xr
import time
import os

In [2]:
ds = xr.open_dataset("cd_ERA5_no-expt_yr_19400101-20220101.nc")
ds

<xarray.Dataset>
Dimensions:       (lat: 721, bnds: 2, lon: 1440, time: 83)
Coordinates:
  * lat           (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon           (lon) float64 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time          (time) datetime64[ns] 1940-01-01 1941-01-01 ... 2022-01-01
    threshold22c  float64 ...
    height2m      float64 ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds      (lat, bnds) float64 ...
    lon_bnds      (lon, bnds) float64 ...
    time_bnds     (time, bnds) datetime64[ns] ...
    cd            (time, lat, lon) float32 ...
    crs           int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         179.875
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2023-10-23 11:44:58.995448+02:00
    tracking_id:                4fbe694e-2dcc-4ecf-8710-695c79ff2511
    project_id:                 c3s-cica-atlas

In [3]:
def fix_atlas_v1(ds):
    var_list = list(ds.coords) + list(ds.data_vars)
    for var in var_list:
        ds[var].encoding["_FillValue"] = None
    for cvar in [
        "member_id",
        "gcm_variant",
        "gcm_model",
        "gcm_institution",
        "rcm_variant",
        "rcm_model",
        "rcm_institution",
    ]:
        for en in ["zlib", "shuffle", "complevel"]:
            try:
                del ds[cvar].encoding[en]
            except KeyError:
                pass
    for var in ds.data_vars:
        # print(var, ds[var].encoding, "\n")
        complevel = ds[var].encoding.get("complevel", 0)
        if complevel > 1:
            ds[var].encoding["complevel"] = 1
            ds[var].encoding["zlib"] = True
            ds[var].encoding["shuffle"] = True
    return ds

In [4]:
ds_fixed = fix_atlas_v1(ds)
ds_fixed

<xarray.Dataset>
Dimensions:       (lat: 721, bnds: 2, lon: 1440, time: 83)
Coordinates:
  * lat           (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon           (lon) float64 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time          (time) datetime64[ns] 1940-01-01 1941-01-01 ... 2022-01-01
    threshold22c  float64 ...
    height2m      float64 ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds      (lat, bnds) float64 ...
    lon_bnds      (lon, bnds) float64 ...
    time_bnds     (time, bnds) datetime64[ns] ...
    cd            (time, lat, lon) float32 ...
    crs           int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         179.875
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2023-10-23 11:44:58.995448+02:00
    tracking_id:                4fbe694e-2dcc-4ecf-8710-695c79ff2511
    project_id:                 c3s-cica-atlas

In [5]:
ds_processed = ds_fixed.isel(time=slice(0, 80))
ds_processed

<xarray.Dataset>
Dimensions:       (lat: 721, bnds: 2, lon: 1440, time: 80)
Coordinates:
  * lat           (lat) float64 -90.0 -89.75 -89.5 -89.25 ... 89.5 89.75 90.0
  * lon           (lon) float64 -180.0 -179.8 -179.5 ... 179.2 179.5 179.8
  * time          (time) datetime64[ns] 1940-01-01 1941-01-01 ... 2019-01-01
    threshold22c  float64 ...
    height2m      float64 ...
Dimensions without coordinates: bnds
Data variables:
    lat_bnds      (lat, bnds) float64 ...
    lon_bnds      (lon, bnds) float64 ...
    time_bnds     (time, bnds) datetime64[ns] ...
    cd            (time, lat, lon) float32 ...
    crs           int32 ...
Attributes: (12/26)
    Conventions:                CF-1.9 ACDD-1.3
    title:                      Copernicus Interactive Climate Atlas: gridded...
    summary:                    Monthly/annual gridded data from observations...
    institution:                Copernicus Climate Change Service (C3S)
    producers:                  Institute of Physics of Cantabria (IFCA, CSIC...
    license:                    CC-BY 4.0, https://creativecommons.org/licens...
    ...                         ...
    geospatial_lon_max:         179.875
    geospatial_lon_resolution:  0.25
    geospatial_lon_units:       degrees_east
    date_created:               2023-10-23 11:44:58.995448+02:00
    tracking_id:                4fbe694e-2dcc-4ecf-8710-695c79ff2511
    project_id:                 c3s-cica-atlas

## Writing with compression level 1

In [6]:
start = time.time()

filename = "atlas_out_level1.nc"
ds_processed.to_netcdf("atlas_out_level1.nc")

duration = time.time() - start
print(f"duration: {duration} secs")

duration: 6.477423191070557 secs


In [7]:
file_size = os.path.getsize(filename)
print("File Size is :", file_size/(1024*1024), "MB")

File Size is : 98.4865894317627 MB
